In [12]:


#IMPORTS


import os
import distutils
from distutils.dir_util import copy_tree
import shutil

import keras
import numpy as np
from PIL import Image


from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input




#Loading and opening files
trainfile= '/Users/kodi/Downloads/food-101/meta/train.txt'
testfile = '/Users/kodi/Downloads/food-101/meta/test.txt'
or_fpath= '/Users/kodi/Downloads/food-101/images/'
trainpath = '/Users/kodi/Downloads/food-101/train/'
testpath = '/Users/kodi/Downloads/food-101/test/'

#CREATING TEST/TRAIN VARIABLES,
traintestfile = [trainfile,testfile]
newdata = []
for file in traintestfile:
    with open(file) as textfile:
        data= textfile.read() 
    newdata.append(str(data).split('\n') )

train = newdata[0]
test = newdata[1]

#INITIALIZE MODEL INPUTS
num_classes= len(classes)  
dropout = .6

batch_size = 32
dimensions = [256,256]
shape = (256,256,3)
epochs = 50
earlystop = EarlyStopping(monitor = 'accuracy', min_delta = 0, patience = 3, verbose = 1)
callbacks = [earlystop]


#TRAIN AND TEST DIRECTORIES\
maketesttraindir()

# CREATE CLASS DIRECTORIES FOR TRAIN AND TEST
makeclassdir(or_fpath)


#SENDS IMAGES INTO TRAIN AND TEST DIRECTORIES
images_to_train_test(or_fpath)

#RESIZE IMAGES to 256 X 256
resizev2([trainpath,testpath])

#REMOVE UNREADABLE IMAGES
removebadfiles(trainpath)


#CREATE CLASSES VARIABLE
with open ('/Users/kodi/Downloads/food-101/meta/classes.txt') as ctgs:
    classes = ctgs.read().splitlines()
    


#CREATE TRAIN AND VALIDATION GENERATORS
train_gen, validation_gen = setup_generator(train_path, test_path, batch_size, dimensions) 

#CREATE FINAL MODEL
final_model = create_model(num_classes, dropout, shape = (256,256,3))

#TRAIN FINAL MODEL
train_model(final_model, train_gen, validation_gen, callbacks)




In [18]:
#FUNCTIONS


#MAKE TEST/ TRAIN DIRECTORY

def maketesttraindir():
    dirnametrain =  '/Users/kodi/Downloads/food-101/train/'
    dirnametest = '/Users/kodi/Downloads/food-101/test/'
    directories = [dirnametrain, dirnametest]
    for direc in directories:
        try:
            if not os.path.exists(direc):
                os.mkdir(direc)

        except:
            print('one already exists')



#CREATING CLASS DIRECTORIES
def makeclassdir(imgpath):
    try:
        for x in os.listdir(imgpath):

                if x.startswith('test'):

                        
                        os.mkdir(testpath+ x.split('st')[1])
                else:
                    os.mkdir(trainpath+ x)
    except:
        pass




#SEND IMAGES INTO RESPECTIVE TEST/TRAIN DIRECTORIES

            
def images_to_train_test(imgpath):
    for filename in os.listdir(or_fpath):   
        try:   
                    for image in os.listdir(or_fpath + filename):

                            if filename.startswith('test'):
                                if os.path.exists('/Users/kodi/Downloads/food-101/'+ 'test/'+ filename.split('test')[1] + '/'+image):
                                    pass
                                else:  
                                    copy_tree('/Users/kodi/Downloads/food-101/images' + '/' +filename +'/','/Users/kodi/Downloads/food-101/'+ 'test/'+ filename.split('test')[1] + '/')                                                                                   
                            else:
                                if os.path.exists('/Users/kodi/Downloads/food-101/'+ 'train/'+ filename +'/'+image):
                                    pass
                                else:
                                    copy_tree('/Users/kodi/Downloads/food-101/images' + '/' +filename +'/','/Users/kodi/Downloads/food-101/'+ 'train/'+ filename + '/')                           
        except:
            pass






#RESIZE IMAGES TO 256 X 256

def resizev2(imgfolders):
    for fold in imgfolders:    
        for foodname in os.listdir(fold): 
            #print(fold + foodname)
            try:


                for item in os.listdir(fold + foodname):
                    #print(fold+foodname+'/'+item)

                          f_img = fold +foodname+"/"+item
                          img = Image.open(f_img)
                          img = img.resize((256,256))
                          img.save(f_img)

            

            except:
                pass
            
            
#REMOVE BAD IMAGE FILES           
            
def removebadfiles(trainpath):
    for foodname in os.listdir(trainpath):
        for image in os.listdir(trainpath + foodname):
            try:
                luv = Image.open(trainpath + foodname + '/'+image)
             
            except:
    
              os.remove(trainpath + foodname + '/'+image)

    
#BUIDLING IMAGE GENERATORS FOR TRAIN AND VALIDATION
    
def setup_gen(trainpath, testpath, batch_size, dimentions):
    from PIL import ImageFile
    ImageFile.LOAD_TRUNCATED_IMAGES = True

    train_datagen =   ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_gen = train_datagen.flow_from_directory(
        trainpath,  
        target_size=dimentions,
        class_mode="categorical",
        color_mode = 'rgb',
        batch_size= batch_size)

    validation_gen = test_datagen.flow_from_directory(
        testpath, 
        class_mode="categorical",
        color_mode = 'rgb',
        target_size=dimentions,
        batch_size=batch_size)

    return train_gen, validation_gen

#CREATE THE RESNET MODEL AND ADD AUGMENTATIONS

def create_model(num_classes, dropout, shape):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_tensor=Input(
            shape=shape))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    final_model = Model(inputs=base_model.input, outputs=predictions)

    return final_model


#TRAIN THE MODEL 

def train_model(final_model, train_gen, validation_gen, callbacks):

    final_model.compile(
        loss='categorical_crossentropy',
        optimizer=SGD(lr=0.0001, momentum=0.9),
        metrics=['accuracy'])

    final_model.fit_generator(train_gen, validation_data=validation_gen,
                              epochs=epochs, callbacks=callbacks,
                              steps_per_epoch=train_gen.samples/batch_size,
                              validation_steps=validation_gen.samples/                             
                              batch_size)








